# 0. Setup

In [1]:
import pytorch3d as p3d
import torch
from starter.utils import get_mesh_renderer

/root/anaconda3/envs/ml3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
renderer = get_mesh_renderer(image_size=512)

In [3]:
vertices = torch.tensor([[[0.0, 0.0, 0.0],
                          [1.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0]]])  # 1 x N_v x 3 tensor.
faces = torch.tensor([[[0, 1, 2]]])  # 1 x N_f x 3 tensor.
textures =  torch.tensor([[[1.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0],
                             [0.0, 0.0, 1.0]]])  # 1 x N_v x 3 tensor.
meshes = p3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=p3d.renderer.TexturesVertex(textures),
)

In [4]:
cameras = p3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0),
    T=torch.tensor([[0.0, 0.0, 3.0]]),
    fov=60.0,
)

In [5]:
lights = p3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]])
rend = renderer(meshes, cameras=cameras, lights=lights)
image = rend[0, ..., :3].numpy()

# 1. Practicing with Cameras (15 Points)

In [14]:
from starter.utils import load_cow_mesh

# renderer = get_mesh_renderer(image_size=256)

# vertices, faces = load_cow_mesh("data/cow.obj")
# vertices = vertices.unsqueeze(0)
# faces = faces.unsqueeze(0)
# textures = torch.ones_like(vertices)
# textures = textures * torch.tensor([0.7, 0.7, 1])
# cow_mesh = p3d.structures.Meshes(
#     verts=vertices,
#     faces=faces,
#     textures=p3d.renderer.TexturesVertex(textures),
# )

cow_mesh = p3d.io.load_objs_as_meshes(["data/cow.obj"])

## 1.1. 360-degree Renders (5 points)

In [17]:
import numpy as np
from tqdm import tqdm

num_images = 36
degrees = np.linspace(0, 360, num=num_images).tolist()
my_images = []

lights = p3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]])

for angle in tqdm(degrees):
    R, T = p3d.renderer.cameras.look_at_view_transform(3.0, 15.0, angle)
    cameras = p3d.renderer.FoVPerspectiveCameras(R=R, T=T, fov=60.0)
    # lights = p3d.renderer.PointLights(location=cameras.get_camera_center())
    rend = renderer(cow_mesh, cameras=cameras, lights=lights)
    image = rend[0, ..., :3].numpy()
    my_images.append((image * 255).astype(np.uint8))

100%|██████████| 36/36 [00:31<00:00,  1.15it/s]


In [18]:
import imageio
duration = 1000 // num_images  # duration per frame in milliseconds
imageio.mimsave('output/cow-360.gif', my_images, duration=duration, loop=0)

Result:

![360-degree renders of the cow](output/cow-360.gif)

## 1.2 Re-creating the Dolly Zoom (10 points)

In [ ]:
# python -m starter.dolly_zoom --duration 30 --num_frames 100

Result:

![Dolly Zoom Effect](output/dolly.gif)

# 2. Practicing with Meshes (10 Points)

## 2.1 Constructing a Tetrahedron (5 points)

## 2.2 Constructing a Cube (5 points)

# 3. Re-texturing a mesh (10 points)

# 4. Camera Transformations (10 points)

# 5. Rendering Generic 3D Representations (45 Points)

## 5.1 Rendering Point Clouds from RGB-D Images (10 points)

## 5.2 Parametric Functions (10 + 5 points)

## 5.3 Implicit Surfaces (15 + 5 points)

# 6. Do Something Fun (10 points)

# (Extra Credit) 7. Sampling Points on Meshes (10 points)